# JPEG 2000

## Uvod

JPEG 2000 je standard za kompresiju slika uveden od strane <b>Joint Photographic Experts Group</b> 2000. godine, kao naslednik njihovog originalnog standarda iz 1992. godine. Glavni napredak u odnosu na prvi standard je upotreba transformacije sa talasićima, koji su u tom periodu počeli da pokazuju izvanredne rezultate u ovoj oblasti. Tek u poslednjih nekoliko meseci su se pojavili uređaji koji enkoduju svoje slike u JPEG 2000 formatu, ali i dalje ih mali broj aplikacija podržava.

U okviru ovog rada obrađene su sledeće faze kompresije:
* <b>Tiling</b> - Podela slike na manje podceline, kako bi se omogućilo paralelno obrađivanje
* <b>Color Transformation</b> - Konverzija iz RGB u YCbCr komponente, radi bolje performanse konverzije
* <b>Wavelet Transformation</b> - Izvlačenje koeficijenata pomoću transformacije talasićima
* <b>Quantization</b> - Uklanjanje koeficijenata koji imaju slab udeo

In [1]:
import os
import sys
import threading
import numpy as np

import queue
import pywt
import image_slicer
# pip install image_slicer
from PIL import Image

## Tiling

Primenom ovog koraka pretprocesiranja dobijamo niz slika manje dimenzije (eng. <i>tiles</i>), na koje se primenjuju dalje operacije.
Proces podele slike nije neophodan (slika može ostati celovita), ali se ovim postupkom štedi na memoriji. Takođe, pruža mogućnost paralelizacije posla. Mana ovog koraka je gubitak kvaliteta, tako što se gomila šum u podslikama sa nižim koeficijentima, koji se kasnije može odraziti na celu sliku, te se sa povećanjem broja podslika smanjuje kvalitet.

Za implementaciju, koristili smo biblioteku <i>image_slicer</i> koja nam, pomoću svojih metoda <i>slice</i> i <i>join</i>, pruža sve potrebne funkcionalnosti. Za razliku od originalne implementacije, koja podrazumeva da koristimo slike određene veličine, ovde navodimo broj podslika, koje su sve iste dimenzije.

In [2]:
"""
Funkcija koja deli sliku.

Parametri:
filename - ime fajla
numberOfTiles - broj podslika

Povratna vrednost:
tuple podslika
"""
def tileImage(filename, numberOfTiles=2):
    return image_slicer.slice(filename, numberOfTiles, save=False)

In [3]:
"""
Funkcija koja spaja delove u jednu sliku

Parametri:
tiles - tuple podslika

Povratna vrednost:
objedinjena slika
"""
def joinTiles(tiles):
    return image_slicer.join(tiles)

## Color encoding

Korak enkodovanja boja, odnosno prebacivanja komponenti iz RGB u YCbCr, se primenjuje jer jer ustanovljeno da se primenom ovakvog koraka poboljšava stepen kompresije. Takođe, ni ovaj postupak nije obavezan.
* Y komponenta predstavlja nivo osvetljenosti
* Cb plava komponenta u odnosu na zelenu komponentu
* Cr crvena komponenta u odnosu na zelenu komponentu

Komponente Cb i Cr su manje primetne ljudskom oku, te su pogodnije za manipulaciju.

Za implementaciju su iskorišćene modifikovane matrice prelaska JPEG konverzije.

In [4]:
"""
Kodiranje boja iz RGB u YCbCr

Parametri:
img - polazna slika

Povratna vrednost:
slika sa YCbCr komponentama
"""
def colorEncode(img):
    imgEncoded = img.convert('RGB')
    
    (width, height) = imgEncoded.size
    M = np.array([[0.299, 0.587, 0.114],
                  [-0.168935, -0.331665, 0.50059],
                  [0.499813, -0.4187, -0.081282]])
    for x in range(width):
        for y in range(height):
            R, G, B = imgEncoded.getpixel((x, y))
            result = M.dot(np.array([R, G, B]).T) + np.array([0, 128, 128])
            
            result[0] = 0 if (result[0] <= 0) else result[0]
            result[0] = 255 if (result[0] >= 255) else result[0]
            result[1] = 0 if (result[1] <= 0) else result[1]
            result[1] = 255 if (result[1] >= 255) else result[1]
            result[2] = 0 if (result[2] <= 0) else result[2]
            result[2] = 255 if (result[2] >= 255) else result[2]
            
            
            imgEncoded.putpixel((x, y), (int(result[0]), int(result[1]), int(result[2])))
            
    return imgEncoded

In [5]:
"""
Dekodiranje boja iz YCbCr u RGB

Parametri:
img - slika sa YCbCr

Povratna vrednost:
slika sa RGB komponentama
"""
def colorDecode(img):
    imgDecoded = img
    
    (width, height) = imgDecoded.size    
    M = np.array([[1.0, 0, 1.402],
                  [1.0, -0.34414, -0.71414],
                  [1.0, 1.772, 0]
                 ])
    for x in range(width):
        for y in range(height):
            Y, Cb, Cr = imgDecoded.getpixel((x, y))
            result = M.dot(np.array([Y, Cb-128, Cr-128]).T)
            
            result[0] = 0 if (result[0] <= 0) else result[0]
            result[0] = 255 if (result[0] >= 255) else result[0]
            result[1] = 0 if (result[1] <= 0) else result[1]
            result[1] = 255 if (result[1] >= 255) else result[1]
            result[2] = 0 if (result[2] <= 0) else result[2]
            result[2] = 255 if (result[2] >= 255) else result[2]
            
            imgDecoded.putpixel((x, y), (int(result[0]), int(result[1]), int(result[2])))
    return imgDecoded

## Wavelet transformation

Primenom metode transformacije talasićima dobijamo matricu koeficijenata. U različitim delovima matrice se nalaze različiti tipovi koeficijenata. U gornjoj levoj četvrtini matrice se nalazi aproksimacija slike, u gornjoj desnoj vertikalne komponente, u donjoj levoj horizontalne, dok se u donjoj desnoj nalaze dijagonalne komponente.

Za implementaciju se koriste dva tipa talasića, CDF 5/3 i 9/7. Oba su iz familije biortogonalnih talasića. CDF 5/3 koristi celobrojne koeficijente, zbog čega u slučaju njihove primene korak kvantizacije nije neophodan. Sa druge strane CDF 9/7 se često karakteriše kao nepovratni, jer ubacuje u signal određenu dozu šuma.

U konkretnoj implementaciji smo koristili biblioteku pywt, koja sadrži metode za rad za transformacijama pomoću talasića. Kako u skupu dostupnih talasića nisu ponuđeni CDF 5/3 i CDF 9/7, primenili smo bior2.2 talasić, koji je iz iste familije i izgledom je najsličniji talasiću CDF 5/3.

In [6]:
"""
Primena wavelet (talasica) transformacije na sliku

Parametri:
img - ulazna slika
level - nivo transformacije

Povratna vrednost:
tuple koeficijenata
"""
def waveTransform(img, level=3):
    (width, height) = img.size
    img = img.copy()
    
    comps1 = np.empty((width, height))
    comps2 = np.empty((width, height))
    comps3 = np.empty((width, height))
    
    for x in range(width):
        for y in range(height):
            comp1, comp2, comp3 = img.getpixel((x, y))
            comps1[x, y] = comp1
            comps2[x, y] = comp2
            comps3[x, y] = comp3

    # Koristimo bior2.2 posto je najslicniji cfd transformaciji
    coefs1 = pywt.wavedec2(comps1, 'bior2.2', level=level)
    coefs2 = pywt.wavedec2(comps2, 'bior2.2', level=level)
    coefs3 = pywt.wavedec2(comps3, 'bior2.2', level=level)
    
    return (coefs1, coefs2, coefs3)

In [7]:
"""
Primena inverzne wavelet transformacije na koeficijente

Parametri:
coefs - tuple wavelet koeficijenata

Povratna vrednost:
originalna slika
"""
def iwaveTransform(coefs):
    (coefs1, coefs2, coefs3) = coefs
    comps1 = pywt.waverec2(coefs1, 'bior2.2')
    comps2 = pywt.waverec2(coefs2, 'bior2.2')
    comps3 = pywt.waverec2(coefs3, 'bior2.2')
    width, height = comps1.shape
    img = Image.new('RGB', (width, height))
    for x in range(width):
        for y in range(height):
            img.putpixel((x, y), (int(comps1[x, y]), int(comps2[x, y]), int(comps3[x, y])))
    return img

In [8]:
"""
Vizuelizacija koeficijenata wavelet transformacije

Parametri:
coefs - tuple wavelet koeficijenata

Povratna vrednost:
slika komponenti wavelet transformacija
"""
def imageFromCoefs(coefs):
    (coefs1, coefs2, coefs3) = coefs
    (width, height) = coefs1[0].shape
    cA1 = np.array(coefs1[0])
    
    cA2 = np.array(coefs2[0])
    
    cA3 = np.array(coefs3[0])
    img_size = (width*2**(np.array(coefs1).size - 1),\
                height*2**(np.array(coefs1).size - 1))
    img = Image.new('RGB', img_size)
    
    for x in range(width):
        for y in range(height):
            img.putpixel((x, y), (int(cA1[x, y]), int(cA2[x, y]), int(cA3[x, y])))
    
    for i in range(1, np.array(coefs1).size):
        
        cH1i = (coefs1[i])[0]
        cV1i = (coefs1[i])[1]
        cD1i = (coefs1[i])[2]
        
        cH2i = (coefs2[i])[0]
        cV2i = (coefs2[i])[1]
        cD2i = (coefs2[i])[2]
        
        cH3i = (coefs3[i])[0]
        cV3i = (coefs3[i])[1]
        cD3i = (coefs3[i])[2]
        
        for x in range(cH1i.shape[0]):
            for y in range(cH1i.shape[1]):
                img.putpixel((width+x, y), (int(cH1i[x, y]), int(cH2i[x, y]), int(cH3i[x, y])))
        
        for x in range(cH1i.shape[0]):
            for y in range(cH1i.shape[1]):
                img.putpixel((x, height+y), (int(cV1i[x, y]), int(cV2i[x, y]), int(cV3i[x, y])))
                
        for x in range(cH1i.shape[0]):
            for y in range(cH1i.shape[1]):
                img.putpixel((width+x, height+y), (int(cD1i[x, y]), int(cD2i[x, y]), int(cD3i[x, y])))
                
        width += cH1i.shape[0]
        height += cH1i.shape[1]
    return img

## Quantization

U ovom koraku je cilj otkloniti koeficijente koji imaju relativno mali uticaj na sliku, po cenu kvaliteta. Ovaj korak zapravo predstavlja kompresiju. Kvantizacija se može vršiti na osnovu kvantizacionog intenziteta ili na osnovu predefinisane matrice, čija se veličina određuje veličinom podslika. Korisno je napomenuti da se različit intenzitet kvantizacije može primeniti na različite podslike. Izbor intenziteta kvantizacije(u intervalu $(0, 1]$) utiče na očuvanje kvaliteta slike.

Kvantizacija se vrši na dva nivoa:
* Deljenje koeficijenata 
* Zaokruživanje koeficijenata na celobrojne vrednosti

Koeficijenti koji su manji od intenziteta kvantizacije postaju 0 i na taj način se gubi njihova informacija. Što je intenzitet veći, to je veći broj 0 u matrici, te je i veći stepen kompresije, ali i veći gubitak kvaliteta.

In [9]:
"""
Pomocna funkcija za kvantizaciju jednog nivoa

Parametri:
coefs - tuple wavelet koeficijenata
delta_b - intenzitet kvantizacije

Povratna vrednost:
tuple kvantizovanih koeficijenata
"""
def quantizeCoefs(coefs, delta_b):
    coefsH = coefs[0]
    coefsV = coefs[1]
    coefsD = coefs[2]
    
    for x in range(coefsH.shape[0]):
        for y in range(coefsH.shape[1]):
            coefsH[x, y] = np.sign(coefsH[x,  y])* int(np.abs(coefsH[x, y]) / delta_b)
            coefsV[x, y] = np.sign(coefsV[x,  y])* int(np.abs(coefsV[x, y]) / delta_b)
            coefsD[x, y] = np.sign(coefsD[x,  y])* int(np.abs(coefsD[x, y]) / delta_b)
    
    return (coefsH, coefsV, coefsD)
    

In [10]:
"""
Primena kvantizacije

Parametri:
coefs - tuple wavelet koeficijenata
delta_b - intenzitet kvantizacije (0,1]
inverse - da li je inverzna kvantizacija

Povratna vrednost:
tuple kvantizovanih koeficijenata
"""
def quantization(coefs, delta_b=1, inverse=False):
    if delta_b <= 0 or delta_b > 1:
        sys.exit("Neispravan intenzitet konverzije.")
    
    if not inverse:
        delta_b = 1/delta_b
        
    coefs1 = coefs[0]
    coefs2 = coefs[1]
    coefs3 = coefs[2]
    
    cA1 = np.array(coefs1[0])
    
    cA2 = np.array(coefs2[0])
    
    cA3 = np.array(coefs3[0])
    
    for x in range(cA1.shape[0]):
        for y in range(cA1.shape[1]):
            cA1[x, y] = np.sign(cA1[x, y]) * int(np.abs(cA1[x, y])/delta_b)
            cA2[x, y] = np.sign(cA2[x, y]) * int(np.abs(cA2[x, y])/delta_b)
            cA3[x, y] = np.sign(cA3[x, y]) * int(np.abs(cA3[x, y])/delta_b)
    
    coefs1[0] = cA1
    coefs2[0] = cA2
    coefs3[0] = cA3
    
    for i in range(1, np.array(coefs1).size):
        coefs1[i] = quantizeCoefs(coefs1[i], delta_b)
        
        coefs2[i] = quantizeCoefs(coefs2[i], delta_b)
        
        coefs3[i] = quantizeCoefs(coefs3[i], delta_b)

    return (coefs1, coefs2, coefs3)

## Testing

In [11]:
"""
Poziv vizualizacije
"""
def visualizeWavelet(filename, level=3):
    img = Image.open(filename)
    imageFromCoefs(waveTransform(img, level)).show()    

In [12]:
"""
Posao jedne niti
"""
def processTile(tile, level, quant, my_queue):
    tile.image = quantization(waveTransform(colorEncode(tile.image), level), quant)
    tile.image = colorDecode(iwaveTransform(quantization(tile.image, quant, inverse=True)))

    my_queue.put((tile,))


In [13]:
"""
Konvertera
"""
def runJ2KConverter(filename, numberOfTiles=2, level=3, quant=1, save=False):
    tiles = tileImage(filename, numberOfTiles)

    jobs = []
    my_queue = queue.Queue()

    for i in range(len(tiles)):
        job = threading.Thread(target=processTile, args=[tiles[i], level, quant, my_queue])
        jobs.append(job)
        
    for i in range(len(tiles)):
        jobs[i].start()

    for i in range(len(tiles)):
        jobs[i].join()

    tiles = ()
    while not my_queue.empty():
        tiles = tiles + my_queue.get()

    
    img = joinTiles(tiles)
    
    img.show();
    
    if save:
        img.save('converted_image.jpg')

In [14]:
runJ2KConverter('lena.png', numberOfTiles=8, quant=1, save=True)

In [15]:
visualizeWavelet('marko_u_svemiru.jpg', level=1)

## Literatura

* https://www.spiedigitallibrary.org/journals/optical-engineering/volume-53/issue-12/123102/JPEG-2000-based-compression-of-fringe-patterns-for-digital-holographic/10.1117/1.OE.53.12.123102.full?SSO=1
* http://cs.haifa.ac.il/~nimrod/Compression/Wavelets/w6jpeg2k-1.pdf
* http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.591.4975&rep=rep1&type=pdf
* https://www.mathworks.com/help/wavelet/gs/introduction-to-the-wavelet-families.html
* https://en.wikipedia.org/wiki/JPEG_2000